In [ ]:
###############
## Libraries ##
###############

import sys,os
import datetime as dt
import itertools as itt

import numpy  as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot   as plt
import matplotlib.gridspec as mplg
import matplotlib.animation as mpla
import cartopy.crs as ccrs

import SDFC as sd
import scipy.stats as sc
import scipy.interpolate as sci
import statsmodels.api as sm
from multiprocessing import Pool, freeze_support

from statsmodels.gam.api import GLMGam, BSplines

path = "./data/"

### Matplotlib
plt.rcParams['text.usetex'] = True
plt.rcParams['xtick.top'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['xtick.major.width'] = 0.5
plt.rcParams['ytick.major.width'] = 0.5


#############
## Classes ##
#############

from class_gev import GEVFixedBoundLinearLink , GEVFixedBoundLinearLink_alt , GEVFixedBoundStationaryLink , GEVNonStationaryLogExp 
from class_gev import GEVNonStationaryLogExp_loc , GEVNonStationaryLogExp_scale , GEVNonStationaryLogExp_shape


In [ ]:
# Model M0 - Plot stationary (mu, sigma, xi, bound, return level, return time, cdf1, cdf2)

## Parameters GEV
mu = 23
sigma = 1.35
xi = - 0.15

## Parameters fit
n_size_tab = [50, 100, 200]
n_times = 1000
n_cov = 100

## Load data
fits_n_size_err0 = [xr.load_dataarray(path + "fits_m0_gev_data_size" + str(n) + "_err0.nc") for n in n_size_tab]
metrics_n_size_err0 = [xr.load_dataarray(path + "metrics_m0_gev_data_size" + str(n) + "_err0.nc") for n in n_size_tab]
fits_n_size_err25 = [xr.load_dataarray(path + "fits_m0_gev_data_size" + str(n) + "_err25.nc") for n in n_size_tab]
metrics_n_size_err25 = [xr.load_dataarray(path + "metrics_m0_gev_data_size" + str(n) + "_err25.nc") for n in n_size_tab]
fits_n_size_err5 = [xr.load_dataarray(path + "fits_m0_gev_data_size" + str(n) + "_err5.nc") for n in n_size_tab]
metrics_n_size_err5 = [xr.load_dataarray(path + "metrics_m0_gev_data_size" + str(n) + "_err5.nc") for n in n_size_tab]

fits_tab = [fits_n_size_err0,fits_n_size_err25,fits_n_size_err5]
metrics_tab = [metrics_n_size_err0,metrics_n_size_err25,metrics_n_size_err5]

for id_err,err in enumerate([0,25,5]):
    fits = fits_tab[id_err]
    metrics = metrics_tab[id_err]


    tab_letters = ['a','b','c','d','e','f','g','h','i']
    fig = plt.figure(figsize=(7,3.5), dpi=1200)

    tab_param = [mu,sigma,xi]
    tab_param_label = [r"$\hat{\mu}$",r"$\hat{\sigma}$",r"$\hat{\xi}$"]
    for id_param,param in enumerate(["loc","scale","shape"]):
        ax = plt.subplot(2,4,1+id_param)
        ax.boxplot(fits[0].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1-0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "white", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        ax.boxplot(fits[1].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "lightblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        ax.boxplot(fits[2].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1+0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "royalblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)

        ax.axhline(tab_param[id_param], color='black', linewidth=0.5, linestyle='dashed')
        ax.set_ylabel(tab_param_label[id_param])
        ax.set_xticks(np.arange(fits[0]['models'].size)+1)
        ax.set_xticklabels("")
        ax.tick_params(axis='y', labelsize=8)
        ax.tick_params(axis='x', labelsize=8)        

        ax.grid(linestyle='dotted', linewidth=0.3)
        ax.text(0.97, 0.97, tab_letters[id_param], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')

    tab_metric = [0,0,0]
    tab_metric_label = [r"$\hat{B} - B$",r"$\hat{z}_{p=1\%} - z_{p=1\%}$",r"$\hat{r}(z_{p=1\%}) - 100$",r"$\|\hat{F} - F\|_\infty$",r"$\|\hat{F} - F\|_2$"]
    for id_metric,metric in enumerate(["result_bounds","return_level_100y","return_time_100y","cdf_norm_Linf","cdf_norm_L2"]):
        ax = plt.subplot(2,4,4+id_metric)
        bplot1 = ax.boxplot(metrics[0].sel(metrics=metric).values.T, positions=np.arange(fits[0]['models'].size)+1-0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "white", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        bplot2 = ax.boxplot(metrics[1].sel(metrics=metric).values.T, positions=np.arange(fits[0]['models'].size)+1, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "lightblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        bplot3 = ax.boxplot(metrics[2].sel(metrics=metric).values.T, positions=np.arange(fits[0]['models'].size)+1+0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "royalblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)

        ax.set_xticks(np.arange(fits[0]['models'].size)+1)
        ax.set_ylabel(tab_metric_label[id_metric])
        if id_metric < 3:
            ax.axhline(tab_metric[id_metric], color='black', linewidth=0.5, linestyle='dashed')
        if id_metric >= 1:
            ax.set_xticklabels([ r'$\mathcal{M}_0$' , r'$\mathcal{M}_0^\mu$' , r'$\mathcal{M}_0^\sigma$' , r'$\mathcal{M}_0^\xi$' , r'$\mathcal{M}_0^B$' ])
        else:
            ax.set_xticklabels("")
        if id_metric == 2:
            ax.set_ylim(-100,800)
        if id_metric == 0:
            ax.set_ylim(-10,10)
            ax.set_ylabel(tab_metric_label[id_metric], labelpad=-3)
        if id_metric in [3,4]:
            ax.ticklabel_format(axis='y', scilimits=[-1, 0])
            ax.set_ylim(bottom=0)
        if id_metric == 4:
            ax.ticklabel_format(axis='y', scilimits=[-1, 0])
            ax.set_ylim(bottom=0)
        ax.tick_params(axis='y', labelsize=8)        
        ax.tick_params(axis='x', labelsize=8)        

        ax.grid(linestyle='dotted', linewidth=0.3)
        ax.text(0.97, 0.97, tab_letters[4 + id_metric], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')

    ax.legend([bplot1["boxes"][0], bplot2["boxes"][0], bplot3["boxes"][0]], [r'$n=50$', r'$n=100$', r'$n=200$'], loc='center', bbox_to_anchor=(-1.75, -0.3), fancybox=True, ncol=3)
    fig.subplots_adjust( left = 0.08 , right = 0.99 , bottom = 0.16 , top = 0.98 , hspace = 0.2 , wspace = 0.5 )
    plt.show()

In [ ]:
# Model M1 - Plot stationary (mu0, mu1, sigma, xi, bound, return level, return time, cdf1, cdf2)

## Parameters GEV
mu0 = 23
mu1 = 1.6
sigma = 1.35
xi = - 0.15

## Parameters fit
n_size_tab = [50, 100, 200]
n_times = 1000
n_cov = 100

## Load data
fits_n_size_err0 = [xr.load_dataarray(path + "fits_m1_gev_data_size" + str(n) + "_err0.nc") for n in n_size_tab]
metrics_n_size_err0 = [xr.load_dataarray(path + "metrics_m1_gev_data_size" + str(n) + "_err0.nc") for n in n_size_tab]
fits_n_size_err25 = [xr.load_dataarray(path + "fits_m1_gev_data_size" + str(n) + "_err25.nc") for n in n_size_tab]
metrics_n_size_err25 = [xr.load_dataarray(path + "metrics_m1_gev_data_size" + str(n) + "_err25.nc") for n in n_size_tab]
fits_n_size_err5 = [xr.load_dataarray(path + "fits_m1_gev_data_size" + str(n) + "_err5.nc") for n in n_size_tab]
metrics_n_size_err5 = [xr.load_dataarray(path + "metrics_m1_gev_data_size" + str(n) + "_err5.nc") for n in n_size_tab]

fits_tab = [fits_n_size_err0,fits_n_size_err25,fits_n_size_err5]
metrics_tab = [metrics_n_size_err0,metrics_n_size_err25,metrics_n_size_err5]

for id_err,err in enumerate([0,25,5]):
    fits = fits_tab[id_err]
    metrics = metrics_tab[id_err]

    tab_letters = ['a','b','c','d','e','f','g','h','i']
    fig = plt.figure(figsize=(7,7), dpi=1200)

    tab_param = [mu0,mu1,sigma,xi]
    tab_param_label = [r"$\hat{\mu}_0$",r"$\hat{\mu}_1$",r"$\hat{\sigma}$",r"$\hat{\xi}$"]
    for id_param,param in enumerate(["loc0","loc1","scale","shape"]):
        ax = plt.subplot(3,3,1+id_param)
        ax.boxplot(fits[0].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1-0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "white", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        ax.boxplot(fits[1].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "lightblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        ax.boxplot(fits[2].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1+0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "royalblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)

        ax.axhline(tab_param[id_param], color='black', linewidth=0.5, linestyle='dashed')
        ax.set_ylabel(tab_param_label[id_param])
        ax.set_xticks(np.arange(fits[0]['models'].size)+1)
        ax.set_xticklabels("")
        ax.tick_params(axis='y', labelsize=8)        
        ax.tick_params(axis='x', labelsize=8) 
        
        ax.grid(linestyle='dotted', linewidth=0.3)
        ax.text(0.97, 0.97, tab_letters[id_param], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')

    tab_metric = [0,0,0]
    tab_metric_label = [r"$\hat{B} - B$",r"$\hat{z}_{p=1\%} - z_{p=1\%}$",r"$\hat{r}(z_{p=1\%}) - 100$",r"$\|\hat{F} - F\|_\infty$",r"$\|\hat{F} - F\|_2$"]
    for id_metric,metric in enumerate(["result_bounds","return_level_100y","return_time_100y","cdf_norm_Linf","cdf_norm_L2"]):
        ax = plt.subplot(3,3,5+id_metric)
        bplot1 = ax.boxplot(metrics[0].sel(metrics=metric).values.reshape(fits[0]['models'].size,n_times*n_cov).T, positions=np.arange(fits[0]['models'].size)+1-0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "white", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        bplot2 = ax.boxplot(metrics[1].sel(metrics=metric).values.reshape(fits[0]['models'].size,n_times*n_cov).T, positions=np.arange(fits[0]['models'].size)+1, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "lightblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        bplot3 = ax.boxplot(metrics[2].sel(metrics=metric).values.reshape(fits[0]['models'].size,n_times*n_cov).T, positions=np.arange(fits[0]['models'].size)+1+0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "royalblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)

        ax.set_xticks(np.arange(fits[0]['models'].size)+1)
        ax.set_ylabel(tab_metric_label[id_metric])
        if id_metric < 3:
            ax.axhline(tab_metric[id_metric], color='black', linewidth=0.5, linestyle='dashed')
        if id_metric >= 2:
            ax.set_xticklabels([ r'$\mathcal{M}_1$' , r'$\mathcal{M}_1^\mu$' , r'$\mathcal{M}_1^\sigma$' , r'$\mathcal{M}_1^\xi$' , r'$\mathcal{M}_1^B$' ])
        else:
            ax.set_xticklabels("")
        if id_metric == 2:
            ax.set_ylim(-100,800)
        if id_metric == 0:
            ax.set_ylim(-10,10)
            ax.set_ylabel(tab_metric_label[id_metric], labelpad=-3.5)
        if id_metric in [3,4]:
            ax.ticklabel_format(axis='y', scilimits=[-1, 0])
            ax.set_ylim(bottom=0)
        if id_metric == 4:
            ax.ticklabel_format(axis='y', scilimits=[-1, 0])
            ax.set_ylim(bottom=0)
        ax.tick_params(axis='y', labelsize=8)        
        ax.tick_params(axis='x', labelsize=8) 

        ax.grid(linestyle='dotted', linewidth=0.3)
        ax.text(0.97, 0.97, tab_letters[4 + id_metric], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')

    ax.legend([bplot1["boxes"][0], bplot2["boxes"][0], bplot3["boxes"][0]], [r'$n=50$', r'$n=100$', r'$n=200$'], loc='center', bbox_to_anchor=(-0.8, -0.25), fancybox=True, ncol=3)
    fig.subplots_adjust( left = 0.08 , right = 0.99 , bottom = 0.1 , top = 0.99 , hspace = 0.125 , wspace = 0.3 )
    plt.show()


In [ ]:
# Model M2 - (mu0, mu1, sigma0, sigma1, xi, bound, return level, return time, cdf1, cdf2)

## Parameters GEV
mu0 = 23
mu1 = 1.6
sigma0 = 1.35
sigma1 = 0.1
xi = - 0.15

## Parameters fit
n_size_tab = [50, 100, 200]
n_times = 1000
n_cov = 100

## Load data
fits_n_size_err0 = [xr.load_dataarray(path + "fits_m2_gev_data_size" + str(n) + "_err0.nc") for n in n_size_tab]
metrics_n_size_err0 = [xr.load_dataarray(path + "metrics_m2_gev_data_size" + str(n) + "_err0.nc") for n in n_size_tab]
fits_n_size_err25 = [xr.load_dataarray(path + "fits_m2_gev_data_size" + str(n) + "_err25.nc") for n in n_size_tab]
metrics_n_size_err25 = [xr.load_dataarray(path + "metrics_m2_gev_data_size" + str(n) + "_err25.nc") for n in n_size_tab]
fits_n_size_err5 = [xr.load_dataarray(path + "fits_m2_gev_data_size" + str(n) + "_err5.nc") for n in n_size_tab]
metrics_n_size_err5 = [xr.load_dataarray(path + "metrics_m2_gev_data_size" + str(n) + "_err5.nc") for n in n_size_tab]

fits_tab = [fits_n_size_err0,fits_n_size_err25,fits_n_size_err5]
metrics_tab = [metrics_n_size_err0,metrics_n_size_err25,metrics_n_size_err5]

for id_err,err in enumerate([0,25,5]):
    fits = fits_tab[id_err]
    metrics = metrics_tab[id_err]

    tab_letters = ['a','b','c','d','e','f','g','h','i','j']
    fig = plt.figure(figsize=(7,7./5.*2), dpi=1200)

    tab_param = [mu0,mu1,sigma0,sigma1,xi]
    tab_param_label = [r"$\hat{\mu}_0$",r"$\hat{\mu}_1$",r"$\hat{\sigma}_0$",r"$\hat{\sigma}_1$",r"$\hat{\xi}$"]
    for id_param,param in enumerate(["loc0","loc1","sigma0","sigma1","shape"]):
        ax = plt.subplot(2,5,1+id_param)
        ax.boxplot(fits[0].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1-0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "white", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        ax.boxplot(fits[1].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "lightblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        ax.boxplot(fits[2].sel(params=param).T, positions=np.arange(fits[0]['models'].size)+1+0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "royalblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)

        ax.axhline(tab_param[id_param], color='black', linewidth=0.5, linestyle='dashed')
        ax.set_ylabel(tab_param_label[id_param], fontsize=8)
        ax.set_xticks(np.arange(fits[0]['models'].size)+1)
        ax.set_xticklabels("")
        ax.tick_params(axis='y', labelsize=8)        
        ax.tick_params(axis='x', labelsize=6) 
        if id_param == 4:
            ax.set_ylabel(tab_param_label[id_param], labelpad=-3.5, fontsize=8)
        
        ax.grid(linestyle='dotted', linewidth=0.3)
        ax.text(0.97, 0.97, tab_letters[id_param], fontsize=12, transform=ax.transAxes, fontweight='bold', va='top', ha='right')

    tab_metric = [0,0,0]
    tab_metric_label = [r"$\hat{B} - B$",r"$\hat{z}_{p=1\%} - z_{p=1\%}$",r"$\hat{r}(z_{p=1\%}) - 100$",r"$\|\hat{F} - F\|_\infty$",r"$\|\hat{F} - F\|_2$"]
    for id_metric,metric in enumerate(["result_bounds","return_level_100y","return_time_100y","cdf_norm_Linf","cdf_norm_L2"]):
        ax = plt.subplot(2,5,6+id_metric)
        bplot1 = ax.boxplot(metrics[0].sel(metrics=metric).values.reshape(fits[0]['models'].size,n_times*n_cov).T, positions=np.arange(fits[0]['models'].size)+1-0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "white", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        bplot2 = ax.boxplot(metrics[1].sel(metrics=metric).values.reshape(fits[0]['models'].size,n_times*n_cov).T, positions=np.arange(fits[0]['models'].size)+1, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "lightblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)
        bplot3 = ax.boxplot(metrics[2].sel(metrics=metric).values.reshape(fits[0]['models'].size,n_times*n_cov).T, positions=np.arange(fits[0]['models'].size)+1+0.25, showcaps=False, patch_artist=True, boxprops = dict(facecolor = "royalblue", linewidth=0.3), medianprops = dict(color = "black", linewidth = 0.5), whiskerprops = dict(linewidth=0.3), showfliers=False, widths=0.2)

        ax.set_xticks(np.arange(fits[0]['models'].size)+1)
        ax.set_ylabel(tab_metric_label[id_metric], fontsize=8)
        if id_metric < 3:
            ax.axhline(tab_metric[id_metric], color='black', linewidth=0.5, linestyle='dashed')
        if id_metric >= 0:
            ax.set_xticklabels([ r'$\mathcal{M}_1$' , r'$\mathcal{M}_1^\mu$' , r'$\mathcal{M}_1^\sigma$' , r'$\mathcal{M}_1^\xi$' , r'$\mathcal{M}_1^B$' ], fontsize = 8)
        else:
            ax.set_xticklabels("")
        if id_metric == 2:
            ax.set_ylim(-100,800)
        if id_metric == 0:
            ax.set_ylim(-10,10)
            ax.set_ylabel(tab_metric_label[id_metric], labelpad=-3.5, fontsize=8)
        if id_metric == 1:
            ax.set_ylabel(tab_metric_label[id_metric], labelpad=-1, fontsize=8)
        if id_metric in [3,4]:
            ax.ticklabel_format(axis='y', scilimits=[-1, 0])
            ax.set_ylim(bottom=0)
        if id_metric == 4:
            ax.ticklabel_format(axis='y', scilimits=[-1, 0])
            ax.set_ylim(bottom=0)
        ax.tick_params(axis='y', labelsize=8)        
        ax.tick_params(axis='x', labelsize=6) 

        ax.grid(linestyle='dotted', linewidth=0.3)
        ax.text(0.97, 0.97, tab_letters[5 + id_metric], fontsize=12, transform=ax.transAxes, fontweight='bold', va='top', ha='right')

    ax.legend([bplot1["boxes"][0], bplot2["boxes"][0], bplot3["boxes"][0]], [r'$n=50$', r'$n=100$', r'$n=200$'], loc='center', bbox_to_anchor=(-2.8, -0.4), fancybox=True, ncol=3)
    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.2 , top = 0.99 , hspace = 0.2 , wspace = 0.52 )
    plt.show()